In [90]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.compose import make_column_selector

# Encoders 
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split




In [96]:
df = pd.read_csv("Customer-Churn-Records.csv")


df = df.drop(columns=['RowNumber','CustomerId'])
df = df.rename(columns={'Card Type': 'CardType'})
df = df.rename(columns={'Point Earned': 'PointsEarned'})
df = df.rename(columns={'Geography': 'Country'})
# save original columns
myColumns = df.columns
# df.head()


In [97]:
from sklearn.model_selection import train_test_split

# Split the dataset into training, temporary holdout, and final holdout sets
X_train, X_temp, y_train, y_temp = train_test_split(df.drop('Exited', axis=1), df['Exited'], test_size=0.2, stratify=df['Exited'], random_state=42)

# Split the temporary holdout set into validation and final holdout sets
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X_val.shape)
print(y_val.shape)

(8000, 15)
(8000,)
(1000, 15)
(1000,)
(1000, 15)
(1000,)


In [98]:
X_train.head()

,Surname,CreditScore,Country,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Complain,Satisfaction Score,CardType,PointsEarned
4791,Taylor,709,Spain,Male,35,2,0.00,2,1,0,104982.39,0,2,GOLD,422
8881,Kornilova,744,France,Male,29,1,43504.42,1,1,1,119327.75,0,1,PLATINUM,607
6166,Kodilinyechukwu,773,France,Male,64,2,145578.28,1,0,1,186172.85,0,1,SILVER,630
4473,Wang,646,Germany,Female,29,4,105957.44,1,1,0,15470.91,0,1,PLATINUM,345
854,Baresi,675,France,Female,57,8,0.00,2,0,1,95463.29,0,3,SILVER,632


In [66]:
df.head()

,Surname,CreditScore,Country,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,CardType,PointsEarned
0,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1,1,2,DIAMOND,464
1,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,1,3,DIAMOND,456
2,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,1,3,DIAMOND,377
3,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0,5,GOLD,350
4,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,0,5,GOLD,425


<font color='0FF00'> 
Let us feature transform each feature now! 
There are several different encodings and transformations we could try.
Some of them, such as target encoding, are more experimental in nature.
</font>

<font color='skyBlue'>Banks usually categorize credit scores into several risk profiles or categories. According to CNBC 1, credit scores are typically divided into five categories:

Deep subprime: Credit scores below 580
Subprime: Credit scores between 580 and 619
Near-prime: Credit scores between 620 and 659
Prime: Credit scores between 660 and 719
Super-prime: Credit scores of 720 or above

These categories help banks assess the creditworthiness of borrowers and determine the types of financial products and interest rates they are eligible for..</font>

In [67]:
#
#! CreditScore



# Create a dictionary mapping credit score ranges to ordinal values
encoding_dict = {
    (0, 580): 0,
    (581, 619): 1,
    (620, 659): 2,
    (660, 719): 3,
    (720, 10000): 4
}

# Define a function to apply the ordinal encoding
def apply_encoding(credit_score):
    for score_range, encoded_value in encoding_dict.items():
        if score_range[0] <= credit_score <= score_range[1]:
            return encoded_value

# Apply ordinal encoding using apply() function
df['CreditScoreOrd'] = df['CreditScore'].apply(apply_encoding)

# Print the DataFrame
# df.head()


In [68]:
for i, j in encoding_dict.items():
    print(i, "XXX", j)

(0, 580) XXX 0
(581, 619) XXX 1
(620, 659) XXX 2
(660, 719) XXX 3
(720, 10000) XXX 4


In [69]:
#
# ! Age

#* Only difference is that the countries have different retirement ages.
#* I also considered binning the age where students start university.
age_groups = {
    'France': [(18, 24), (25, 35), (36, 45), (46, 62), (63 , 100)],
    'Germany': [(18, 24), (25, 35), (36, 45), (46, 65), (66 , 100)], 
    'Spain': [(18, 24), (25, 35), (36, 45), (46, 66), (67 , 100)],
}

#? https://www.thelocal.de/20230127/how-does-germanys-retirement-age-compare-to-the-rest-of-europe
#? https://www.cleiss.fr/docs/regimes/regime_france/an_3.html
#? https://www.caserexpatinsurance.com/blog-typical-non-spanish/retirement-age-in-spain

def get_age_groups(country, age):
    tax_ranges = age_groups[country]
    for index, tax_range in enumerate(tax_ranges):
        if tax_range[0] <= age <= tax_range[1]:
            return index
    return None

df['AgeOrd'] = df.apply(lambda row: get_age_groups(row['Country'], row['Age']), axis=1)

<font color='skyBlue'> 
I discretize EstimatedSalary salary based on tax brackets that are used in the respective country of the member.

Spain has one more bracket compared to France and Germany, so there are multiple ways to handle this. I just merged the top brackets.
</font>

In [70]:
#
#! EstimatedSalary

tax_brackets = {
    'France': [(0, 10_225), (10_226, 26_070), (26_071, 74_545), (74_546, 160_336), (16_0337, float('inf'))], # For 2022
    'Germany': [(0, 10_347), (10_348, 15_999), (16000, 62809), (62810, 277825), (277826, float('inf'))], # For 2023. Also the source has a gap between the first brackets that i filled in (Up to 10,347. 10,909-15,999)
    'Spain': [(0, 12_450), (12451, 20200), (20201, 35200), (35201, 100000), (100001, float('inf'))], # For 2021. Spain has 6 brackets, so we are combining the highest 2
}

def get_tax_bracket(country, salary):
    tax_ranges = tax_brackets[country]
    for index, tax_range in enumerate(tax_ranges):
        if tax_range[0] <= salary <= tax_range[1]:
            return index
    return None

df['TaxBracket'] = df.apply(lambda row: get_tax_bracket(row['Country'], row['EstimatedSalary']), axis=1)


#? https://www.worldwide-tax.com/france/france-taxes.asp
#? https://www.worldwide-tax.com/spain/spain-taxes.asp
#? https://www.worldwide-tax.com/germany/germany-taxes.asp

<font color='skyBlue'> 
The important thing with Balance is that zero balance gets its own category, since its probable that it correlates uniquely with other features. Does not have to mean that the customer has little money, perhaps they are not using the bank to save any money at all
</font>

In [71]:
#
#! Balance

encoding_dict = {
    (-1, 1): 0,
    (2, 100_000): 1,
    (100_001, 125_000): 2,
    (125_001, 149_999): 3,
    (150_000, 300_000): 4
}


# Define a function to apply the ordinal encoding
def apply_encoding(balance):
    for balance_range, encoded_value in encoding_dict.items():
        if balance_range[0] <= balance <= balance_range[1]:
            return encoded_value

# Apply ordinal encoding using apply() function
df['BalanceOrd'] = df['Balance'].apply(apply_encoding)

<font color='skyBlue'> 
Have not found anything that points earned correlates with any other feature
</font>

In [72]:
#
#! PointsEarned

from sklearn.preprocessing import KBinsDiscretizer

# Assuming your data is in the 'another_column' column of the DataFrame
another_column_subset = df['PointsEarned'].values.reshape(-1, 1)

# Create a KBinsDiscretizer object with 5 bins and the quantile strategy
discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')

# Fit and transform the data using the discretizer
another_column_binned = discretizer.fit_transform(another_column_subset)

# Assign the discretized values to a new column in the DataFrame
df['PointsEarnedQuant'] = another_column_binned.flatten()

<font color='skyBlue'> 
Let us do the same with credit card
</font>

In [73]:
#
#! CardType Credit card type

from sklearn.preprocessing import OrdinalEncoder

# Assuming your data is in the 'another_column' column of the DataFrame
CardTypeValues = df['CardType'].values.reshape(-1, 1)

# Define the order of the cards as a list
card_order = ['SILVER', 'GOLD', 'PLATINUM', 'DIAMOND']

# Create an OrdinalEncoder object with the desired order
encoder = OrdinalEncoder(categories=[card_order])

# Fit and transform the data using the encoder
CardTypeEnc = encoder.fit_transform(CardTypeValues)

# Assign the encoded values to a new column in the DataFrame
df['CardTypeOrd'] = CardTypeEnc.flatten()


In [74]:
df.CardType.value_counts()

CardType
DIAMOND     2507
GOLD        2502
SILVER      2496
PLATINUM    2495
Name: count, dtype: int64

<font color='skyBlue'> 
Even though we did not find a significant correlation between name count and Exited we will try frequency encoding, since there could be some more complex relationships that the models could uncover
</font>

In [75]:
#
#* Ordinal encoding of Surname based on name count (only the 10 most common names), less common names get encoded as 0

# Calculate the frequency of occurrence for each value in the 'Surname' column
value_frequencies = df['Surname'].value_counts()

# Select the top 10 most common names
top_names = value_frequencies.nlargest(11).index

# Reverse index so that most common name gets encoded as the highest value
top_names = top_names[::-1]

# Create a dictionary mapping the top names to their ordinal values (0 to 9)
encoding_dict = {name: i for i, name in enumerate(top_names)}

# Use the dictionary to map the 'Surname' column values to their ordinal values in a new column,
# assigning 0 to the remaining names

df['SurnameOrd'] = df['Surname'].map(encoding_dict).fillna(0)
# df['SurnameOrd'] = df['Surname'].map(encoding_dict).fillna(50)

<font color='skyBlue'> Purchasing power parities (PPPs) are the rates of currency conversion that try to equalise the purchasing power of different currencies, by eliminating the differences in price levels between countries. The basket of goods and services priced is a sample of all those that are part of final expenditures: final consumption of households and government, fixed capital formation, and net exports. This indicator is measured in terms of national currency per US dollar.   </font>

In [76]:
#
#* Let us try to ordinal encode country based on PPP.
countryPPP = {'Germany': 0.913, 'Spain':0.742, 'France': 0.901}
#* Let us just get the order from this rather than teh PPP value
countryPPP = {'Germany': 3, 'Spain':1, 'France': 2}
#! Could be a bad idea since we are pretty sure that Germans are leaving the bank the most.

df['CountryOrd'] = df['Country'].map(countryPPP)


#? https://data.oecd.org/conversion/purchasing-power-parities-ppp.htm

<font color='skyBlue'> 
While we are at it. Le us try to add some wellbeing scores for each country since happiness is highly correalted with your economy (more on the lower end though)
</font>

In [77]:
#
#* Let us try to ordinal encode country based on PPP.
countryHappyScore = {'Germany': 7.03, 'Spain':6.48, 'France': 6.69}
#* Let us just get the order from this rather than teh PPP value
countryHappyScoreOrd = {'Germany': 3, 'Spain':1, 'France': 2}

df['CountryHappy'] = df['Country'].map(countryHappyScore)

#! Since the countries will get the same ordinal scores as they did with the PPP encoding it will not be able to uncover any none-linear patterns. So we either encode a different way (e.g. proportional or keep the happy number) or dont add this feature it all

#? https://wisevoter.com/country-rankings/happiest-countries-in-the-world/


In [78]:
#
#* Gender is the only binary feature that is in string format
df['GenderBinary'] = df['Gender'].apply(lambda x: 0 if x == 'Male' else 1)


<font color='skyBlue'> 
Tenure and NumofProducts are ordinal by default so no need to transorm (to clarify you dont need to ordinal encode since, usually one-hot is good option)

HasCrCard, IsActiveMember, Complain are already label encoded (which is also one-hot in this case). Also our target feature Exited does also not need a transformation.
</font>

In [79]:
#* The original features/columns
myColumns

Index(['Surname', 'CreditScore', 'Country', 'Gender', 'Age', 'Tenure',
       'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'EstimatedSalary', 'Exited', 'Complain', 'Satisfaction Score',
       'CardType', 'PointsEarned'],
      dtype='object')

In [80]:
df.head()

,Surname,CreditScore,Country,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,...,CreditScoreOrd,AgeOrd,TaxBracket,BalanceOrd,PointsEarnedQuant,CardTypeOrd,SurnameOrd,CountryOrd,CountryHappy,GenderBinary
0,Hargrave,619,France,Female,42,2,0.00,1,1,1,...,1,2,3,0,1.0,3.0,0.0,2,6.69,1
1,Hill,608,Spain,Female,41,1,83807.86,1,0,1,...,1,2,4,1,1.0,3.0,0.0,1,6.48,1
2,Onio,502,France,Female,42,8,159660.80,3,1,0,...,0,2,3,4,1.0,3.0,0.0,2,6.69,1
3,Boni,699,France,Female,39,1,0.00,2,0,0,...,3,2,3,0,0.0,1.0,0.0,2,6.69,1
4,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,...,4,2,3,3,1.0,1.0,0.0,1,6.48,1


<font color='skyBlue'> 
Let us end this stage by dropping the features that have been transformed
</font>

In [81]:
#
#* We are dropping every feature that has been transformed. The features that dont need transformation are kept
df_trans = df.drop(['Surname', 'CreditScore', 'Country', 'Age', 'Balance', 'EstimatedSalary', 'CardType', 'PointsEarned', 'Gender'], axis=1)

#* saving the transformed dataFrame
df_trans.to_csv('df_trans.csv', index=False)

In [82]:
df_trans.head(20)

,Tenure,NumOfProducts,HasCrCard,IsActiveMember,Exited,Complain,Satisfaction Score,CreditScoreOrd,AgeOrd,TaxBracket,BalanceOrd,PointsEarnedQuant,CardTypeOrd,SurnameOrd,CountryOrd,CountryHappy,GenderBinary
0,2,1,1,1,1,1,2,1,2,3,0,1.0,3.0,0.0,2,6.69,1
1,1,1,0,1,0,1,3,1,2,4,1,1.0,3.0,0.0,1,6.48,1
2,8,3,1,0,1,1,3,0,2,3,4,1.0,3.0,0.0,2,6.69,1
3,1,2,0,0,0,0,5,3,2,3,0,0.0,1.0,0.0,2,6.69,1
4,2,1,1,1,0,0,5,4,2,3,3,1.0,1.0,0.0,1,6.48,1
5,8,2,1,0,1,1,5,2,2,4,2,1.0,3.0,0.0,1,6.48,0
6,7,2,1,1,0,0,2,4,3,0,0,0.0,0.0,0.0,2,6.69,0
7,4,4,1,0,1,1,2,0,1,3,2,0.0,3.0,0.0,3,7.03,1
8,4,2,0,1,0,0,3,0,2,3,3,0.0,1.0,0.0,2,6.69,0
9,2,1,1,1,0,0,3,3,1,2,3,0.0,1.0,0.0,2,6.69,0


In [83]:
df_trans.columns

Index(['Tenure', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'Exited',
       'Complain', 'Satisfaction Score', 'CreditScoreOrd', 'AgeOrd',
       'TaxBracket', 'BalanceOrd', 'PointsEarnedQuant', 'CardTypeOrd',
       'SurnameOrd', 'CountryOrd', 'CountryHappy', 'GenderBinary'],
      dtype='object')

In [85]:

df_trans.isnull().sum()

Tenure                0
NumOfProducts         0
HasCrCard             0
IsActiveMember        0
Exited                0
Complain              0
Satisfaction Score    0
CreditScoreOrd        0
AgeOrd                0
TaxBracket            0
BalanceOrd            0
PointsEarnedQuant     0
CardTypeOrd           0
SurnameOrd            0
CountryOrd            0
CountryHappy          0
GenderBinary          0
dtype: int64